In [17]:
%run /home/analysis_user/New_trap_code/Tools/basic_packages.py
from matplotlib.ticker import MaxNLocator


plt.rcParams["figure.figsize"] = (16,12)
plt.rcParams["font.size"] = 24
plt.rcParams['xtick.labelsize'] = "small"
plt.rcParams['ytick.labelsize'] = 36

In [18]:
ll = likelihood_analyser.LikelihoodAnalyser()
bandwidth = 2
decimate = 10
drive_freq = 71
fsamp = 5000
def get_scale(i,phase="sin"):
    calibrationFile = step_cal_files[i]       
    calibFreqs = np.fft.rfftfreq(len(calibrationFile.electrode_data[0]), d=1./5000)
    fft_angles = np.angle(np.fft.rfft(calibrationFile.electrode_data[0]))
    if(phase=="cos"):
        phaseResponse = (fft_angles[calibFreqs==drive_freq])%(2*np.pi)
    elif(phase=="sin"):
        phaseResponse = (fft_angles[calibFreqs==drive_freq]+np.pi/2)%(2*np.pi)

    print(phaseResponse,np.std(calibrationFile.electrode_data[0])*np.sqrt(2))

    fit_kwargs = {'A': 0, 'f': drive_freq, 'phi': 0, 
                  'error_A': 1, 'error_f': 1, 'error_phi': 0.5, 'errordef': 1,
                  'limit_phi': [-2 * np.pi, 2 * np.pi], 
                  'limit_A': [-1, 1], 
                  'print_level': 0, 'fix_f': True, 'fix_phi': True}

    frequency = fit_kwargs['f']

    b, a = signal.butter(3, [2.*(drive_freq-bandwidth/2.)/fsamp, 2.*(drive_freq+bandwidth/2.)/fsamp ], btype = 'bandpass')
    xx2 = signal.filtfilt(b, a, calibrationFile.x2)[::decimate]
    
    m1_tmp = ll.find_mle_sin(xx2, fsamp=5000/decimate, noise_rms=1, plot=False, suppress_print=True, **fit_kwargs)
    
    return m1_tmp.values[0]

In [19]:
step_cal_dir = r'/data/new_trap/20210819/Bead1/Discharge/SingleResponse/Standard/plus_x/'
step_cal_files = load_dir(step_cal_dir, start_file=0)

100%|██████████| 10/10 [00:00<00:00, 24.36it/s]

10  files in folder
10  files loaded


In [20]:
dischargeAmps1 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    dischargeAmps1.append(get_scale(i))

0 / 10
[0.08164256] 9.996291358974252
1 / 10
[0.08164295] 9.996287299207635
2 / 10
[0.08165177] 9.996293767291393
3 / 10
[0.08164753] 9.996289228249696
4 / 10
[0.08165847] 9.996295047758263
5 / 10
[0.08166351] 9.996289603073315
6 / 10
[0.08165357] 9.996288997930739
7 / 10
[0.08165966] 9.996294070431219
8 / 10
[0.08167838] 9.996291808624145
9 / 10
[0.08168261] 9.996289283900033


In [21]:
step_cal_dir = r'/data/new_trap/20210819/Bead1/Discharge/SingleResponse/Standard/minus_x/'
step_cal_files = load_dir(step_cal_dir, start_file=0)

100%|██████████| 10/10 [00:00<00:00, 32.45it/s]

10  files in folder
10  files loaded


In [22]:
dischargeAmps2 = []
for i in range(len(step_cal_files)):
    print(i,'/',len(step_cal_files))
    dischargeAmps2.append(get_scale(i))

0 / 10
[0.4638042] 0.00011956572544555545
1 / 10
[0.67151998] 0.00012114562185569542
2 / 10
[0.36529674] 0.00012174467681560128
3 / 10
[0.51317175] 0.00012061712473132875
4 / 10
[0.21729539] 0.0001212888449510136
5 / 10
[0.46225922] 0.00012042140508266521
6 / 10
[0.59511334] 0.00011913911478909998
7 / 10
[0.41071241] 0.00011982192811102266
8 / 10
[0.57382034] 0.00012023469865109958
9 / 10
[0.38062756] 0.00011983432330420194


In [24]:
np.mean(dischargeAmps1)/np.mean(dischargeAmps2)

0.7399796420933165